In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
sys.path.append('/home/manu/TFG_repo/scripts')  # make modules in scripts folder callable from notebooks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
sys.path.append('/home/manu/TFG_repo/scripts')

In [ ]:
from config import data_dir
from analysis_functions import *
plt.style.use('/home/manu/TFG_repo/notebooks/notebook.mplstyle')

In [14]:
temp, pres, date = load_time_series('AGL_20181116_chain.nc')

In [15]:
df_fit = load_SHDR_fit('AGL_1_SB56_fit.csv')

In [18]:
df_fit['D1'].std()

52.21645812765153

In [34]:
result_1 = df_fit.loc[10, ['D1', 'b2', 'c2', 'b3', 'a2', 'a1']].to_numpy()

In [37]:
def get_fit_limits(z, y, max_b2_c2=0.5, min_depth=10):
    z = np.abs(z) # in case heights are defined negative

    min_z, max_z = z.min(), z.max()
    min_y, max_y = y.min(), y.max()
    
    lims = np.array([[1.0, max_z],    # D1
            [0.0, max_b2_c2],    # b2
            [0.0, max_b2_c2],    # c2
            [0.0 if max_z < min_depth else - abs((max_y - min_y) / (max_z - min_z)), 0.0], # b3
            [0.0, max_y - min_y],     # a2
            [min_y, max_y]])          # a1

    lims_min = lims[:, 0]
    lims_max = lims[:, 1]
    return lims_min, lims_max

In [56]:

result_1[1]=0.45
v_min, v_max = 0.85 * result_1, 1.15 * result_1
lims_min, lims_max = get_fit_limits(pres[10], temp[10])

TypeError: cannot unpack non-iterable int object

In [57]:
for i in range(6):
    lim_min_d = min(v_min[i], v_max[i])
    lim_max_d = max(v_min[i], v_max[i])
    lims_min[i] = max(lims_min[i], lim_min_d)
    lims_max[i] = min(lims_max[i], lim_max_d)
    lims_delta = (lims_min, lims_max)

In [58]:
print((lims_delta[0] >= lims_min).all())
print((lims_delta[1] <= lims_max).all())
print(lims_delta[1])

True
True
[ 4.51531572e+01  5.00000000e-01  0.00000000e+00 -3.35060537e-03
  3.71665993e+00  1.56284000e+01]


In [61]:
lims_min_delta, lims_max_delta = 0.85 * result_1, 1.15 * result_1
for i in np.where(np.sign(result_1) < 0)[0]:
    lims_min_delta[i], lims_max_delta[i] = lims_max_delta[i], lims_min_delta[i]


lims_min = np.where(lims_min_delta >= lims_min, lims_min_delta,  lims_min)
lims_max = np.where(lims_max_delta <= lims_max, lims_max_delta, lims_max)
print(lims_min, lims_max)

[33.374072731016625 0.3825 0.0 -0.00453317196888264 2.7470964684316934
 13.242116000542358] [45.15315722431661 0.5 0.0 -0.00335060536830456 3.716659927878173 15.6284]


In [59]:
print(lims_min, lims_max)

[ 3.33740727e+01  3.82500000e-01  0.00000000e+00 -4.53317197e-03
  2.74709647e+00  1.32421160e+01] [ 4.51531572e+01  5.00000000e-01  0.00000000e+00 -3.35060537e-03
  3.71665993e+00  1.56284000e+01]
